In [1]:
import sklearn
import numpy as np

In [2]:
from sklearn.datasets import load_iris
datasets = load_iris()

In [3]:
x_data = datasets["data"]
x_data[:5]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

In [20]:
y_data = datasets["target"]
y_data

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [21]:
y_data = y_data.reshape([-1,1])   # 2-dim array로 변환
y_data[:3]

array([[0],
       [0],
       [0]])

In [25]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit(y_data)  
y_data = enc.transform(y_data).toarray()
y_data[-1]

C:\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


array([0., 0., 1.])

In [26]:
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()
x_data_minmax = min_max_scaler.fit_transform(x_data)
x_data_minmax[:3]

array([[0.22222222, 0.625     , 0.06779661, 0.04166667],
       [0.16666667, 0.41666667, 0.06779661, 0.04166667],
       [0.11111111, 0.5       , 0.05084746, 0.04166667]])

In [27]:
x_0 = np.ones(x_data_minmax.shape[0])
x_data_minmax = np.column_stack((x_0, x_data_minmax))   # np.column_stack(): np.hstack()과 같음

x_data_minmax[:3]

array([[1.        , 0.22222222, 0.625     , 0.06779661, 0.04166667],
       [1.        , 0.16666667, 0.41666667, 0.06779661, 0.04166667],
       [1.        , 0.11111111, 0.5       , 0.05084746, 0.04166667]])

In [28]:
weights = np.random.uniform(size=(3,5))
weights

array([[0.57233675, 0.68420495, 0.07374767, 0.3553486 , 0.46951416],
       [0.46341101, 0.84848403, 0.05684121, 0.2924814 , 0.51989555],
       [0.66433926, 0.66451405, 0.77414444, 0.47222218, 0.78133467]])

In [29]:
def softmax(z):
    e = np.exp(z)
    p = e / np.sum(np.exp(z), axis=1).reshape([-1,1])
    return p

In [30]:
z = x_data_minmax.dot(weights.T)
z.shape

(150, 3)

In [31]:
softmax(z[:5])

array([[0.27433249, 0.25194027, 0.47372724],
       [0.2934909 , 0.26802853, 0.43848056],
       [0.28677582, 0.25942759, 0.45379659],
       [0.29043809, 0.26117151, 0.4483904 ],
       [0.27081948, 0.2474073 , 0.48177323]])

In [33]:
np.sum(softmax(z[:5]), axis=1)   # softmax이기 때문에 각 행의 합은 1이다

array([1., 1., 1., 1., 1.])

In [34]:
def cross_entropy_function(y, x, weights):
    z = x_data_minmax.dot(weights.T)
    result = - np.sum(
                np.sum(
                    (y * np.log(softmax(z))), axis=1).reshape((-1,1))
                )
    return result 

In [35]:
cross_entropy_function(y_data,x_data_minmax,weights)

165.9146074468527

In [36]:
def minimize_grdient(y, x, initial_weights, iterations = 500000, alpha=0.001):
    cost_history= []
    theta_history = []
    m = y.shape[0]
    theta = np.copy(initial_weights)
    
    number_of_classes = theta.shape[0]
    number_of_weights = theta.shape[1]
    
    for _ in range(iterations):
        original_theta = np.copy(theta)
        for k in range(number_of_classes):        
            for j in range(number_of_weights):
                partial_x = x[:, j]
                partial_entropy = y - softmax(x.dot(original_theta.T))
                theta[k][j]  = original_theta[k][j] + (
                            alpha * partial_entropy[:,k].dot(partial_x.T) ) / 150
        if (_ % 10000) == 0:
            print(cross_entropy_function(y, x, theta) / 150)
            cost_history.append(cross_entropy_function(y, x, theta))
    return theta, cost_history

In [37]:
# weights = minimize_grdient(y_data, x_data_minmax,weights)
theta, cost_history = minimize_grdient(y_data, x_data_minmax, weights)

1.106018676382896
0.7469399691949831
0.6076849948463545
0.5332135171534659
0.4853137709597411
0.4508891068929888
0.42435380464799877
0.4029150580752561
0.3850072765347514
0.3696755486494297
0.3562989604906477
0.3444531749822433
0.3338364716223568
0.324227279919895
0.315458470772947
0.30740109516758435
0.2999537131685551
0.29303516940506735
0.28657956919820426
0.2805327029604859
0.2748494491067414
0.2694918534954167
0.26442768623353996
0.25962934147199745
0.2550729876801082
0.25073790354372
0.24660595326932955
0.24266116786658992
0.23888940790277913
0.23527808953899296
0.23181596019107853
0.22849291345301603
0.22529983534284906
0.22222847573160578
0.21927134016702843
0.21642159832788696
0.21367300612729462
0.21101983908653674
0.20845683506932847
0.20597914483281043
0.2035822891402025
0.20126212140884508
0.1990147950499217
0.19683673480261957
0.1947246114837056
0.19267531966939852
0.19068595790460896
0.18875381109873024
0.18687633481987703
0.1850511412431329


In [38]:
rand_index = np.random.randint(0, 150, 30)
rand_index

array([132,  75, 118,  71,  51,  46,  19,  12,  63,   7,   1,   9,  56,
        91,   0,  30,  62,  63, 100,  81,  83,  71,  92,  34,  62,  73,
        72,  80,  75,  63])

In [39]:
y_pred = np.argmax(softmax(x_data_minmax[rand_index].dot(theta.T)), axis=1) 
y_pred

array([2, 1, 2, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 2, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1], dtype=int64)

In [40]:
y_true = np.argmax(y_data[rand_index], axis=1)
y_true

array([2, 1, 2, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 2, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1], dtype=int64)

In [41]:
y_pred == y_true

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [43]:
# Accuracy
sum(y_pred == y_true) / len(rand_index)

1.0